## Домашнее задание

### Задание 1
Напишите функцию, которая возвращает название валюты (поле ‘Name’) с максимальным значением курса с помощью сервиса   
https://www.cbr-xml-daily.ru/daily_json.js

In [8]:
def find_highest_currency():
        import requests
        full_data = requests.get('https://www.cbr-xml-daily.ru/daily_json.js').json()
        highest_currency = 0
        highest_currency_Name = ''
        for element in full_data['Valute']:

            if full_data['Valute'][element]['Value'] > highest_currency: 
                highest_currency = full_data['Valute'][element]['Value']
                highest_currency_Name = element
                
        return highest_currency_Name  
        
print(find_highest_currency())

XDR


### Задание 2
Добавьте в класс Rate параметр diff (со значениями True или False), который в случае значения True в методах курсов валют (eur, usd итд) будет возвращать не курс валюты, а изменение по сравнению в прошлым значением. Считайте, self.diff будет принимать значение True только при возврате значения курса. При отображении всей информации о валюте он не используется.

In [79]:
from exchange import Rate
import requests

class Rate_(Rate):
    def __init__(self, format='value', diff = False):
        super().__init__(format)
        if (diff == True) and (self.format == 'value'):
            self.diff = diff
        else: self.diff = False
    
    def differential_exchange_rates(self):
        """
        Возвращает ответ сервиса с информацией об изменении в валюте по сравнению с прошлым значением в виде:
        
        -0.452
        
        """
        today = requests.get('https://www.cbr-xml-daily.ru/daily_json.js').json()
        yesterday = requests.get('https:'+today['PreviousURL'].strip('daily_json.js')+'/'+'daily_json.js').json()
        a = round( today['Valute'][self.currency]['Value'] - yesterday['Valute'][self.currency]['Value'], 3)
        return a
    
    def make_format(self, currency):
        #super().make_format(currency) #почему то эта функция не работает
        """
        Возвращает информацию о валюте currency в двух вариантах:
        - полная информация о валюте при self.format = 'full':
        Rate_('full').make_format('EUR')
        {
            'CharCode': 'EUR',
            'ID': 'R01239',
            'Name': 'Евро',
            'Nominal': 1,
            'NumCode': '978',
            'Previous': 79.6765,
            'Value': 79.4966
        }
        
        Rate_('value').make_format('EUR')
        79.4966
        
        Rate_('value',diff = True ).make_format('EUR')
        -0.452
        """
        self.currency = currency
        if self.diff == True:
            resp = self.differential_exchange_rates()
            return resp
        
        response = self.exchange_rates()
        
        if currency in response:
            if self.format == 'full':
                return response[currency]
            elif self.diff == True:
                response = self.differential_exchange_rates()
                return response
            if self.format == 'value':
                return response[currency]['Value']
        
        return 'Error'
   
a = Rate('value')
print(a.make_format('EUR'))

b = Rate_(format='value', diff = False)
print(b.make_format('EUR'))

c = Rate_(format='value', diff = True)
print(c.make_format('USD'))

d = Rate_()
print(d.make_format('EUR'))


# today = requests.get('https://www.cbr-xml-daily.ru/daily_json.js').json()
# print('https:'+today['PreviousURL'].strip('daily_json.js')+'/'+'daily_json.js')
# yesterday = requests.get('https:'+today['PreviousURL'].strip('daily_json.js')+'/'+'daily_json.js').json()
# print(yesterday)


92.131
92.131
1.145
92.131


### Задание 3
Напишите класс Designer, который учитывает количество международных премий. Подсказки в коде занятия в разделе “Домашнее задание задача 3”.  
### Комментарий по классу Designer такой:
Напишите класс Designer, который учитывает количество международных премий для дизайнеров (из презентации: “Повышение на 1 грейд за каждые 7 баллов. Получение международной премии – это +2 балла”). Считайте, что при выходе на работу сотрудник уже имеет две премии и их количество не меняется со стажем (конечно если хотите это можно вручную менять).

Класс Designer пишется по аналогии с классом Developer из материалов занятия. Комментарий про его условия Вика написала выше: “Повышение на 1 грейд за каждые 7 баллов. Получение международной премии – это +2 балла”

In [119]:
class Employee:
    def __init__ (self, name, accreditation):
        self.name = name
        self.accreditation = accreditation
        
        self.grade = 1
        
    def grade_up(self):
        """This Function will increase an Employee grade"""
        self.grade += 1
        print('Congratulations! Employee {0} has been promoted! New grade is {1} !'.format(self.name, self.grade))
        
    def publish_grade(self):
        """Publish Employee grade and name"""
        print('Employee name is {0}. Grade is {1}'.format(self.name, self.grade) )
    
    def accreditation(self):
        pass
    
class Designer(Employee):
    def __init__(self, name, accreditation, prizes = 2 , gender = 'm'): # prizes - это международная премия
        super().__init__(name, accreditation)
        self.prizes = prizes
        self.points = accreditation + prizes*2  # Новая переменная - будет считать количество очков сотрудника
        
        if gender == 'w': self.promotion_points = 5
        else: self.promotion_points = 7
            
        self.grade = 1 + self.points // self.promotion_points 
        
    def accreditation_passed(self):
        """For each passed accreditation self.points will be increased for 1 point """
        self.accreditation += 1
        self.points +=1
        self.check_for_upgrade()
    
    def prize_recieved(self):
        """for each new prize self.points will be increased for 2 points """
        print('Congratulations! Employee {} recieved prize!'.format(self.name) )
        self.prizes += 1
        self.points += 2
        self.check_for_upgrade()
        
    def check_for_upgrade(self):
        if self.points % self.promotion_points == 0 or (self.points // self.promotion_points + 1) != self.grade:
            self.grade_up()
        return self.publish_all_info()
        
    def publish_all_info(self):
        """Will publish all Employee info"""
        print('Employee {0}, prizes {4} , accreditations {2}, points {1}, grade {3}'.format(self.name,self.points, self.accreditation, self.grade, self.prizes) )
        print()
        
Anna = Designer('Anna',0,2,'w')

Anna.publish_all_info()

Anna.prize_recieved()
Anna.accreditation_passed()

Bob = Designer('Bob',0,2,'m')

Bob.publish_all_info()
Bob.prize_recieved()
Bob.accreditation_passed()

Employee Anna, prizes 2 , accreditations 0, points 4, grade 1

Congratulations! Employee Anna recieved prize!
Congratulations! Employee Anna has been promoted! New grade is 2 !
Employee Anna, prizes 3 , accreditations 0, points 6, grade 2

Employee Anna, prizes 3 , accreditations 1, points 7, grade 2

Employee Bob, prizes 2 , accreditations 0, points 4, grade 1

Congratulations! Employee Bob recieved prize!
Employee Bob, prizes 3 , accreditations 0, points 6, grade 1

Congratulations! Employee Bob has been promoted! New grade is 2 !
Employee Bob, prizes 3 , accreditations 1, points 7, grade 2

